In [13]:
#General libraries
import re
import os
import json

#Fine tune libs
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

#infrence lib
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [14]:
#converting json to txt
prompt_json = r"C:\Projects\GPTChatbot_rag\Prompt_data\prompt_dataset.json"
with open(prompt_json, 'r') as prompts:
    df = json.load(prompts)
    
prompt_txt = []
for conversations in df:
    # f"{conversations['user']}\n{conversations['bot']}\n\n
    # print(conversations['user'])
    user = conversations['user']
    bot = conversations['bot']
    prompt_txt.append(user)
    prompt_txt.append(bot)


In [15]:
prompt_txt_file = r"C:\Projects\GPTChatbot_rag\Prompt_data\prompt_txt.txt"
with open(prompt_txt_file, 'w') as prompt_txt_file:
    for line in prompt_txt:
        try:
            prompt_txt_file.write(line)
            prompt_txt_file.write('\n')
        except:
            pass
print("Prompts are written to txt file successfully!")

Prompts are written to txt file successfully!


In [16]:
def read_documents_from_directory(directory):
    combined_text = ""
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if filename.endswith(".txt"):
            with open(file_path, "r") as file:
                text = file.read()
                combined_text += text
    return combined_text

# Read documents from the directory
train_directory = r'C:\Projects\GPTChatbot_rag\Prompt_data'
text_data = read_documents_from_directory(train_directory)
text_data = re.sub(r'\n+', '\n', text_data).strip()  # Remove excess newline characters

print(text_data)

Hi
Hi!
Hello
Hello!
Hey there
Hi
Good morning
Hi
Hi
Hi
Good afternoon
Hi
Greetings
Hi
Yo
Hi
Hey
Hi
Hi there
Hi
Hello there
Hi
Hiya
Hi
Good day
Hi
Hey!
Hi
Hi!
Hi
Hey there!
Hi
Good morning!
Hi
Hey!
Hi
Good evening!
Hi
Hi!
Hi
Hello there!
Hi
Good day!
Hi
How's it going!
Hi
Hey!
Hi
Hi!
Hi
Hello!
Hi
Hey there!
Hi
Good morning!
Hi
Howdy!
Hi
Greetings!
Hi
Yo!
Hi
What is your address?
Our address is
Address?
Our address is
and your address?
Our address is
give me your address
Our address is
what is your location
Our address is
your telephone?
Our address is
what is your number
Our address is
What food do you have today?
Today we have
Do you have any vegetarian options?
Yes, we have vegetarian foods that are
Are there any gluten-free options available?
Yes, we offer gluten-free food that are
Can I see the dessert menu?
Sure, our dessert options include
Do you offer any beverages?
Yes, we have a variety of drinks including
any beverages?
Yes, we have a variety of drinks including
What drinks do

In [17]:
with open(r"C:\Projects\GPTChatbot_rag\Prompt_data\train.txt", "w") as train_data:
    train_data.write(text_data)
print("Train data created successfully!")

Train data created successfully!


In [18]:
def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset

In [19]:
def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, 
        mlm=mlm,
    )
    return data_collator

In [20]:
def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir)
      
  model = GPT2LMHeadModel.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )
      
  trainer.train()
  trainer.save_model()
     

In [21]:
train_file_path = r"C:\Projects\gptchatbot_rag\prompt_data\train.txt"
model_name = 'gpt2'
output_dir = r'C:\Projects\gptchatbot_rag\outputFineTune'
overwrite_output_dir = True
per_device_train_batch_size = 12
num_train_epochs = 100
save_steps = 60000

In [22]:

train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps,
)

c:\Users\hsohr\miniconda3\Lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
c:\Users\hsohr\miniconda3\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


  0%|          | 0/200 [00:00<?, ?it/s]

{'train_runtime': 1066.2269, 'train_samples_per_second': 1.97, 'train_steps_per_second': 0.188, 'train_loss': 0.37909980773925783, 'epoch': 100.0}


In [23]:
def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model

def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer

def generate_text(model_path, sequence, max_length):
    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    
    # Remove the user input from the sequence

    
    ids = tokenizer.encode(sequence, return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        pad_token_id=model.config.eos_token_id,
        top_k=100,
        early_stopping=True,  
        num_return_sequences=1,  
        max_length=max_length,
    )
    generated_text = tokenizer.decode(final_outputs[0], skip_special_tokens=True)
    
    # Remove the bot prefix "[bot]: " from the generated text
    # Define a regular expression pattern to match "[bot]:"
    pattern = r'\[bot\]:'
    # Replace the matched pattern with an empty string
    generated_text = re.sub(pattern, '', generated_text)
    generated_text = generated_text.split("\n")
    generated_text = generated_text[1]
    return generated_text



In [25]:
model2_path = "C:\\Projects\\gptchatbot_rag\\outputFineTune"
sequence2 = "Hello"
max_len = 20
generate_text(model2_path, sequence2, max_len)


'Hello!'